### 기본 세팅

In [1]:
import time
from datetime import date, timedelta
import pandas as pd

from selenium import webdriver

In [2]:
# 어제 날짜 저장
def get_yesterday():
    today     = date.today()             # 오늘 날짜
    yesterday = today - timedelta(1)     # 어제 날짜

    year = str(yesterday.year)
    mon = str(yesterday.month)
    if int(mon) < 10:
        mon = '0' + str(mon)
    day = str(yesterday.day)
    if int(day) < 10:
        day = '0' + str(day)
        
    return year+mon+day

### 네이버 뉴스 크롤링

In [3]:
def get_navernews_info(keyword):
    
    # 크롤링 정보(크롤링 날짜, 카테고리)
    yesterday = get_yesterday()
    categories = [259, 258, 261, 771, 310, 263]
    
    list_headline_concat    = []     # 모든 카테고리 제목
    list_address_concat     = []     # 모든 카테고리 주소 리스트
    list_comment_num_concat = []     # 모든 카테고리 댓글개수 리스트
    comment_headline_concat = []     # 모든 카테고리 댓글모을 제목 리스트
    comment_comment_concat  = []     # 모든 카테고리 제목별 댓글 리스트
    
    
    
    # 드라이버 실행
    driver = webdriver.Chrome('chromedriver.exe')
    driver.implicitly_wait(3)
    sleeptime = 0.5     # 프로세스 일시정지 시간
    
    
    
    # 카테고리를 돌며 크롤링
    for category in categories:
        page = 0
        list_headline = []     # 제목 리스트
        list_address  = []     # 주소 리스트
        top_headline = ''              # 페이지내 맨위 뉴스 제목
        top_headline_flag = False     # 다음 페이지 맨위 뉴스제목이 기존 페이지 맨위 뉴스제목과 다름  



        # 제목 및 주소 크롤링
        while True:
            try:
                # 페이지 이동
                page += 1
                driver.get(f"https://news.naver.com/main/list.naver?mode=LS2D&sid2={category}&sid1=101&mid=shm&date={yesterday}&page={page}")
                time.sleep(sleeptime)
                
                
                
                # 제목 크롤링
                current = driver.find_elements_by_css_selector("#main_content > div > ul > li > dl > dt > a")
                cnt = 0
                drop_address  = []     # 키워드 미포함 주소 리스트
                for headline in current:
                    # 다음 페이지 맨위 뉴스제목이 기존 페이지 맨위 뉴스제목과 같으면 해당 카테고리 종료
                    if cnt == 1 and headline.text == top_headline:
                        top_headline_flag = True
                        break
                    
                    # 페이지내 맨위 뉴스제목 저장
                    if cnt == 1:
                        top_headline = headline.text

                    # 키워드 있는 제목만 저장
                    if keyword in headline.text:
                        list_headline.append(headline.text)

                    # 키워드 없는 제목 인덱스 저장
                    if keyword not in headline.text:
                        drop_address.append(cnt)
                    cnt += 1
                    
                # 다음 페이지 맨위 뉴스제목이 기존 페이지 맨위 뉴스제목과 같으면 해당 카테고리 종료
                if top_headline_flag == True:
                    break

                    
                    
                # 주소 크롤링
                cnt = 0
                for address in current:
                    # 키워드 없는 제목인 경우 주소 저장 안함
                    if cnt in drop_address:
                        cnt += 1
                        continue

                    # 키워드 있는 제목만 주소 저장
                    if address.get_attribute("href") not in list_address:
                        list_address.append(address.get_attribute("href"))
                    cnt += 1
            except:
                pass



        cnt = 0
        list_comment     = []     # 댓글 리스트
        list_comment_num = []     # 댓글개수 리스트

        # 키워드 포함한 게시글 주소를 돌며 크롤링
        for url in list_address:
            try:
                # 페이지 이동
                cnt += 1
                driver.get(url)
                time.sleep(sleeptime)

                
                
                # 댓글개수 크롤링
                current = driver.find_elements_by_css_selector("#articleTitleCommentCount > span")
                for num in current:
                    if num.text != '댓글':
                        list_comment_num.append(num.text)

                        
                        
                # 댓글 있는 경우 
                if list_comment_num[cnt-1] != '':
                    # 댓글 더보기 클릭  
                    try:
                        while driver.find_element_by_xpath('//*[@id="cbox_module"]/div[2]/div[9]/a').text != '':
                            driver.find_element_by_xpath('//*[@id="cbox_module"]/div[2]/div[9]/a').click()
                            time.sleep(sleeptime)
                    except:
                        while driver.find_element_by_xpath('//*[@id="cbox_module"]/div/div[9]/a').text != '':
                            driver.find_element_by_xpath('//*[@id="cbox_module"]/div/div[9]/a').click()
                            time.sleep(sleeptime)    
                        
                    # 댓글 크롤링
                    cur_comment = []
                    current = driver.find_elements_by_css_selector('span.u_cbox_contents')
                    for comment in current:
                        cur_comment.append(comment.text)
                    list_comment.append(cur_comment)

                    
                    
                # 댓글 없는 경우
                if list_comment_num[cnt-1] == '':
                    cur_comment = []
                    list_comment.append(cur_comment)
            except:
                pass  
            
        if len(list_comment_num) == len(list_address)+1:     # 예외처리
            list_comment_num.pop(0)
        if '댓글' in list_comment_num:                       # 예외처리
            list_headline.pop(0)
            list_address.pop(0)
            list_comment_num.pop(0)

    
    
        comment_headline = []     # 댓글모을 제목 리스트
        comment_comment  = []     # 제목별 댓글 리스트

        # 제목별 댓글 모음
        for i in range(len(list_comment_num)):
            cur_headline = list_headline[i]

            # 댓글 없는 경우 해당 안됨
            if list_comment_num[i] == '':
                continue

            for j in range(len(list_comment[i])):
                cur_comment = list_comment[i][j]

                comment_headline.append(cur_headline)
                comment_comment.append(cur_comment)
                
                
                
        # 카테고리 통합
        list_headline_concat.extend(list_headline)     
        list_address_concat.extend(list_address)     
        list_comment_num_concat.extend(list_comment_num)     
        comment_headline_concat.extend(comment_headline)
        comment_comment_concat.extend(comment_comment)
    driver.close()
            
            
            
    navernews_info = pd.DataFrame({'제목': list_headline_concat, '주소': list_address_concat, '댓글수': list_comment_num_concat})
    navernews_comment = pd.DataFrame({'제목': comment_headline_concat, '댓글': comment_comment_concat})    
    return navernews_info, navernews_comment

### 크롤링 예시

In [4]:
navernews_삼성전자_info, navernews_삼성전자_comment = get_navernews_info('삼성전자')

In [5]:
navernews_삼성전자_info

,제목,주소,댓글수
0,"'낸드 1등' 삼성전자, 2분기 점유율 확대…SK하이닉스는 유지",https://news.naver.com/main/read.naver?mode=LS...,2
1,"이재용 삼성전자 부회장, 준법위 故고계현 위원 조문",https://news.naver.com/main/read.naver?mode=LS...,6
2,"폴더블 흥행 삼성전자, 보급형 'S21 FE'까지 출시...애플 '아이폰13' 출시...",https://news.naver.com/main/read.naver?mode=LS...,3
3,장애인 고용 1위는 현대차…2위 삼성전자는 되레 줄어(종합),https://news.naver.com/main/read.naver?mode=LS...,
4,"삼성전자, '사람이 곧 기업' 입사 3년차도 해외연수 기회 [인재가 곧 경쟁력…기업...",https://news.naver.com/main/read.naver?mode=LS...,2
5,"'낸드 1등' 삼성전자, 2분기 점유율 확대…SK하이닉스는 유지",https://news.naver.com/main/read.naver?mode=LS...,4
6,"낸드 2,3위 합병…삼성전자 위협",https://news.naver.com/main/read.naver?mode=LS...,38
7,삼성전자 초대형 투자 발표 직후 대형 반도체 악재 등장했다,https://news.naver.com/main/read.naver?mode=LS...,47
8,너무 떨어진 삼성전자·SK하이닉스 주식 이제 사도 될까,https://news.naver.com/main/read.naver?mode=LS...,21
9,"TSMC의 반도체 20% 인상, 삼성전자 등 파운드리 업계 전반으로 번질까",https://news.naver.com/main/read.naver?mode=LS...,6


In [6]:
navernews_삼성전자_comment

,제목,댓글
0,"'낸드 1등' 삼성전자, 2분기 점유율 확대…SK하이닉스는 유지",어짜피 하이닉스가 허가 안하면 합병못해하이닉스가 키옥시아지분있음
1,"'낸드 1등' 삼성전자, 2분기 점유율 확대…SK하이닉스는 유지",주가는 압도적 꼴등
2,"이재용 삼성전자 부회장, 준법위 故고계현 위원 조문",재용이는어머니가 교육은 제대로 시켯나바 속이야 알수없지만 감옥에서도 의연히 행동하는...
3,"이재용 삼성전자 부회장, 준법위 故고계현 위원 조문",수수한 모습이 아름다워요^^
4,"이재용 삼성전자 부회장, 준법위 故고계현 위원 조문",기업들이 열심히 공부하고 일할때 본인들은 뭐하고 기업탓을 하는지...ㅉㅉ
...,...,...
122,"삼성전자, 구글 손잡고 1위 퀄컴 추격…""픽셀폰에 5G 모뎀 공급""",호재가 있어도 주가는 하락이냐.뭐가문제냐.
123,"삼성전자, 구글 손잡고 1위 퀄컴 추격…""픽셀폰에 5G 모뎀 공급""",문xx : 훠훠훠 이재용이 뷸러드립쉽시요 훠훠훠
124,"삼성전자, 구글 손잡고 1위 퀄컴 추격…""픽셀폰에 5G 모뎀 공급""",삼성 힘내요미친여당 바꿀때까지 잘 참아주세요
125,"'낸드 1등' 삼성전자, 2분기 점유율 확대…SK하이닉스는 유지",어짜피 하이닉스가 허가 안하면 합병못해하이닉스가 키옥시아지분있음
